In [4]:
import glob
import time
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm as tq
from os import scandir
from torch.utils.tensorboard import SummaryWriter


In [70]:
# %load util.py
import torch
from torch.utils.data import Dataset
import sys
import random
import os
import numpy as np
from datetime import datetime as dt
import glob
from collections import Counter
from sgfmill import sgf
from sgfmill import boards
from sgfmill import ascii_boards
from tqdm import tqdm as tq

class GoDataset(Dataset):
  # pos_paths: list of paths to position features
  # labels: list of moves played in a given position
  def __init__(self, pos_paths, label_paths):
        self.label_paths = label_paths
        self.pos_paths = pos_paths

  def __len__(self):
        return len(self.pos_paths)

  def __getitem__(self, idx):
        pos = torch.load(self.pos_paths[idx])
        correctClass= torch.load(self.label_paths[idx])
        label = correctClass[0] * 19 + correctClass[1]
        return pos, label

def getPaths(prefix):
    fullDataPaths = []
    fullLabelPaths = []
    for rank in ['5k', '4k', '3k', '2k', '1k', '1d', '2d', '3d', '4d']:
        data = glob.glob(prefix + "data/" + rank + "/*.pt")
        labels = glob.glob(prefix + "labels/" + rank + "/*.pt")
        for i in tq(range(len(data))):
            assert data[i].rsplit('/', 1)[1] == labels[i].rsplit('/', 1)[1]
        fullDataPaths.extend(data)
        fullLabelPaths.extend(labels)
    return fullDataPaths, fullLabelPaths

def getCorrectCount(pred, y):
    # pred: batch_size x 361
    # y: batch_size x 361
    return (torch.argmax(pred, dim = 1) == torch.argmax(pred, y, dim = 1)).sum()

def buildFeatures(positions):
    # input: 19 x 19 x 3 input: my stones, their stones, empty spots
    # output: 19 x 19 x 32: 
    # 0 - 7 liberties 
    # 8 - 15 self capture size
    # 16 - 23 capture size
    # 24 - 31 liberties after move
    # candidates: open 1 liberty slots for opponent stones
    n = positions.shape[0]

    
    visited = positions[:, :, 2] - 1
    output = torch.zeros((n, n, 32))
    candidates = []
    libertyMap = dict()
    stoneMap = dict()
    colorMap = dict()
    
    def getLiberties(x, y, color, placeholder):
        firstLiberty = None
        liberties = 0
        stoneCount = 1
        visited[x][y] = placeholder
        visited2[x][y] = 1
        if x > 0:
            if visited2[x - 1][y] == 0:
                visited2[x - 1][y] = 1
                if positions[x - 1][y][2] == 1:
                    if liberties == 0:
                        firstLiberty = [x - 1, y]
                    liberties += 1
                elif positions[x - 1][y][color] == 1:
                    l, s, fl = getLiberties(x - 1, y, color, placeholder)
                    if liberties == 0:
                        firstLiberty = fl
                    liberties += l
                    stoneCount += s
        if x < n - 1:
            if visited2[x + 1][y] == 0:
                visited2[x + 1][y] = 1
                if positions[x + 1][y][2] == 1:
                    if liberties == 0:
                        firstLiberty = [x + 1, y]
                    liberties += 1
                elif positions[x + 1][y][color] == 1:
                    l, s, fl = getLiberties(x + 1, y, color, placeholder)
                    if liberties == 0:
                        firstLiberty = fl
                    liberties += l
                    stoneCount += s
        if y > 0:
            if visited2[x][y - 1] == 0:
                visited2[x][y - 1] = 1
                if positions[x][y - 1][2] == 1:
                    if liberties == 0:
                        firstLiberty = [x, y - 1]
                    liberties += 1
                elif positions[x][y - 1][color] == 1:
                    l, s, fl = getLiberties(x, y - 1, color, placeholder)
                    if liberties == 0:
                        firstLiberty = fl
                    liberties += l
                    stoneCount += s
        if y < n - 1:
            if visited2[x][y + 1] == 0:
                visited2[x][y + 1] = 1
                if positions[x][y + 1][2] == 1:
                    if liberties == 0:
                        firstLiberty = [x, y + 1]
                    liberties += 1
                elif positions[x][y + 1][color] == 1:
                    l, s, fl = getLiberties(x, y + 1, color, placeholder)
                    if liberties == 0:
                        firstLiberty = fl
                    liberties += l
                    stoneCount += s
        
        return liberties, stoneCount, firstLiberty
    
    counter = 1
    for i in range(n):
        for j in range(n):
            # visited: -1 if stone is there, 0 if no stone there, counter if visited + stone present
            if visited[i][j] == -1: 
                # visited2: for each loop through connected stones, 1 if visited or liberty has been checked
                visited2 = torch.zeros_like(visited) 
                color = int(positions[i][j][1] == 1) # 0 if your stone, 1 if opponent stone
                l, s, fl = getLiberties(i, j, color, counter)
                libertyMap[counter] = l
                stoneMap[counter] = s
                colorMap[counter] = color # 0 = my color, 1 = their color
                if l == 1 and color == 1:
                    candidates.append((fl[0], fl[1])) # simpleKo candidate: may not be legal move for me to capture
                if l == 1:
                    output[fl[0]][fl[1]][min(s, 8) - 1 + 8 * (color + 1)] = 1 # mark self captures / captures
                counter += 1
    for i in range(n):
        for j in range(n):
            x = int(visited[i][j].item())
            if x == 0: # mark liberties after move
                if i > 0:
                    key = int(visited[i - 1][j].item())
                    if key != 0 and colorMap[key] == 1:
                        l = libertyMap[key]
                        output[i][j][min(l, 8) + 23] = 1
                if i < n - 1:
                    key = int(visited[i + 1][j].item())
                    if key != 0 and colorMap[key] == 1:
                        l = libertyMap[key]
                        output[i][j][min(l, 8) + 23] = 1
                if j > 0:
                    key = int(visited[i][j - 1].item())
                    if key != 0 and colorMap[key] == 1:
                        l = libertyMap[key]
                        output[i][j][min(l, 8) + 23] = 1
                if j < n - 1:
                    key = int(visited[i][j + 1].item())
                    if key != 0 and colorMap[key] == 1:
                        l = libertyMap[key]
                        output[i][j][min(l, 8) + 23] = 1
            else:
                l = libertyMap[x]
                output[i][j][min(l, 8) - 1] = 1 # mark liberty counts for each stone location
            
    return output, candidates 

# 55 features
# 0-2: stone positions
# 3-4: all 0's, all 1's
# 5-12: turn history
# 13 - 36: liberties, self captures, captures
# 37 - 44: liberties after move
# 45: simple ko constraint
# 46 - 54: one hot encoding for rank
def gameToFeatures(game):
    swapColor = {'w': 'b', 'b': 'w'}
    root_node = game.get_root()
    b = boards.Board(19)
    
    rankOneHot = None
    for rankInd, rank in enumerate(['5çº§', '4çº§', '3çº§', '2çº§', '1çº§', '1æ®µ', '2æ®µ', '3æ®µ', '4æ®µ']):        
        if root_node.get("BR") == rank and root_node.get("WR") == rank:
            assert rankOneHot == None
            rankOneHot = rankInd
    assert rankOneHot != None
    
    features = []
    labels = []
    counter = 0
    for node in game.get_main_sequence():
        color, move = node.get_move()
        #print(color, move)
        feature = torch.zeros(19, 19, 55)
        if color == None:
            feature[:, :, 2] = 1
            feature[:, :, 4] = 1
            feature[:, :, 46 + rankOneHot] = 1
        else:
            labels.append([move[0], move[1]])
            b.play(move[0], move[1], color)
            for c, p in b.list_occupied_points():
                if c != color:
                    # my color: c / their color: color (last move made)
                    feature[p[0], p[1], 0] = 1
                else:
                    feature[p[0], p[1], 1] = 1
            feature[:, :, 2] = (feature[:, :, 0] + feature[:, :, 1]) == 0
            feature[:, :, 4] = 1
            feature[move[0], move[1], 5] = 1
            # moves 1-7 history from last feature => 2-8 history of current feature
            feature[:, :, 6:13] = features[-1][:, :, 5:12] 
            feature[:, :, 13:45], candidates = buildFeatures(feature[:, :, :3])
            feature[:, :, 45] = checkSimpleKo(oldb, b, candidates, swapColor[color])
            feature[:, :, 46 + rankOneHot] = 1
        features.append(feature)
        counter += 1
        oldb = b.copy()
    return features[:-1], labels

def checkSimpleKo(past, present, candidates, color):
    n = 19
    output = torch.zeros((n, n))
    for x, y in candidates:
        variation = present.copy()
        try: 
            variation.play(x, y, color)
            if variation.list_occupied_points() == past.list_occupied_points():
                output[x][y] = 1
        except Exception:
            pass
    return output

def filterGame(game, rank):
    board_size = game.get_size()
    if board_size != 19:
        return False
    root_node = game.get_root()
    if root_node.get("BR") != rank:
        return False
    if root_node.get("WR") != rank:
        return False
    if root_node.get("RU") != "Japanese":
        return False
    if root_node.get("TM") != 600:
        return False
    if root_node.get("KM") != 0:
        return False
    if dt.strptime(root_node.get("DT"), '%Y-%m-%d').year != 2017:
        return False
    return True 

In [7]:
'''writer = SummaryWriter(log_dir = "logs/test")
x = range(100)
for i in x:
    writer.add_scalar('y=2x', i * 2, i)
writer.close()

In [11]:
ranks = ['5k', '4k', '3k', '2k', '1k', '1d', '2d', '3d', '4d'] 

In [7]:
rank = ranks[0]
data = glob.glob("../cleanedGoData/train/data/" + rank + "/*.pt")
labels = glob.glob("../cleanedGoData/train/labels/" + rank + "/*.pt")

In [8]:
print(len(data), len(labels))

319964 319964


In [12]:
fullDataPaths = []
fullLabelPaths = []
for rank in ranks:
    data = glob.glob("../cleanedGoData/train/data/" + rank + "/*.pt")
    labels = glob.glob("../cleanedGoData/train/labels/" + rank + "/*.pt")
    for i in tq(range(len(data))):
        assert data[i].rsplit('/', 1)[1] == labels[i].rsplit('/', 1)[1]
    print(rank, len(data), len(labels))
    fullDataPaths.extend(data)
    fullLabelPaths.extend(labels)

 37%|███▋      | 119127/319964 [00:00<00:00, 1191214.33it/s]

../cleanedGoData/train/data/5k/316088.pt


100%|██████████| 319964/319964 [00:00<00:00, 1269923.82it/s]


5k 319964 319964


 79%|███████▉  | 251931/319088 [00:00<00:00, 1276348.46it/s]

...

3d 318112 318112


 38%|███▊      | 121376/317397 [00:00<00:00, 1213717.75it/s]

../cleanedGoData/train/data/4d/316088.pt


100%|██████████| 317397/317397 [00:00<00:00, 1192802.56it/s]

4d 317397 317397


In [42]:
len(fullDataPaths)

2885727

In [43]:
len(fullLabelPaths)

2885727

In [124]:
len(fullLabelPaths) / 128 * 1.5 / 60 / 60

9.393642578125

In [13]:
training_data = GoDataset(fullDataPaths, fullLabelPaths)

In [14]:
train_dataloader = DataLoader(training_data, batch_size = 128, shuffle = True, num_workers = 4)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [17]:
X, y = next(iter(train_dataloader))

In [71]:
	print("getting paths")
	trainDPaths, trainLPaths = getPaths("../cleanedGoData/train/")
	valDPaths, valLPaths = getPaths("../cleanedGoData/val/")
	trainDPaths = trainDPaths[:50000]
	trainLPaths = trainLPaths[:50000]
	valDPaths = valDPaths[:50000]
	valLPaths = valLPaths[:50000]
    
	print("building dataset")
	training_data = GoDataset(trainDPaths, trainLPaths)
	val_data = GoDataset(valDPaths, valLPaths)

	print("building dataloader")
	train_loader = DataLoader(training_data, batch_size = 128, shuffle = True, num_workers = 4)
	val_loader = DataLoader(val_data, batch_size = 128, shuffle = True, num_workers = 4)

getting paths


100%|██████████| 30736/30736 [00:00<00:00, 1266976.52it/s]


building dataset
building dataloader


In [72]:
X, y = next(iter(train_loader))

In [76]:
X[y[0]]

tensor([[[0., 0., 1.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.]],

        [[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1., 

In [44]:
X[0].shape

torch.Size([128])

In [46]:
	for t in range(1):
		print("Epoch ", t + 1, "\n-----------------------------")
		with torch.enable_grad(), \
				tq(total=len(train_loader.dataset)) as progress_bar:
			for batch, (X, y) in enumerate(train_loader):
				print(X.shape)
				print(y.shape)
				break

  0%|          | 0/50000 [00:00<?, ?it/s]

Epoch  1 
-----------------------------
torch.Size([128, 19, 19, 55])
torch.Size([128, 361])


  0%|          | 0/50000 [00:07<?, ?it/s]


In [19]:
X

tensor([[[[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.]],

         [[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.]],

         [[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 1.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 

In [191]:
y.shape

torch.Size([128, 361])

In [185]:
testLabel = torch.zeros((19, 19))
testLabel[5, 2] = 1
flatten = torch.nn.Flatten(0, -1)
flatten(testLabel).nonzero(as_tuple=True)[0]

tensor([97])

In [15]:
	if torch.cuda.is_available():
		device = torch.device('cuda:0')
	else:
		device = torch.device('cpu')
	print("using device: ", device)

using device:  cuda:0


In [20]:
batchSize = 0
for batch, (X, y) in tq(enumerate(train_dataloader)):
    print(X)


1it [00:05,  5.82s/it]

tensor([[[[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.]],

         [[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.]],

         [[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.]],

         ...,

         [[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [1., 0., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 0., 

3it [00:06,  1.59s/it]

tensor([[[[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.]],

         [[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.]],

         [[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.]],

         ...,

         [[0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          [0., 0., 1.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 

5it [00:10,  2.02s/it]

...

tensor([[[[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.]],

         [[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [1., 0., 0.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.]],

         [[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [1., 0., 0.,  ..., 1., 0., 0.],
          ...,
          [1., 0., 0.,  ..., 1., 0., 0.],
          [1., 0., 0.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.]],

         ...,

         [[0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          [0., 0., 1.,  ..., 1., 0., 0.],
          ...,
          [0., 0., 1.,  ..., 1., 0., 

12it [00:16,  1.01s/it]

tensor([[[[0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.]],

         [[0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.]],

         [[0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          ...,
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.]],

         ...,

         [[0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          [0., 0., 1.,  ..., 0., 0., 1.],
          ...,
          [0., 1., 0.,  ..., 0., 0., 

12it [00:20,  1.68s/it]


KeyboardInterrupt: 

In [ ]:
# BATCH SIZE 128
# shuffle True, num_workers 2      (2.5?)
# shuffle False, num_workers 2
# shuffle True num_workers 4        (1.3?)
# shuffle False num_workers 4

In [27]:
!ls

README.md	 __pycache__	  environment.yml  models.py  util.py
TrainTest.ipynb  buildDataset.py  layers.py	   train.py


In [29]:
pos1 = torch.load(fullDataPaths[0])

In [92]:
!ls ../cleanedGoData/train/data/

1d  1k	2d  2k	3d  3k	4d  4k	5k


In [108]:
posTotal = []
labelTotal = []
counter = 0
prefix = "../cleanedGoData/train/data/4k/"
for entry in tq(os.scandir(prefix)):
    pos = torch.load(prefix + entry.name)
    label = torch.load(prefix + entry.name)
    posTotal.append(pos)
    labelTotal.append(label) 
    counter += 1
    if counter == 50000:
        break

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [100]:
torch.stack(posTotal).shape

torch.Size([50000, 19, 19, 55])

In [62]:
posTotal = torch.stack(posTotal)

In [56]:
torch.save(posTotal, "test.pt")

In [57]:
torch.load("test.pt").shape

torch.Size([100, 19, 19, 55])

In [60]:
!du -sh test.pt

7.6M	test.pt


In [37]:
len(train_dataloader.dataset)

2885727

In [18]:
!ls ../cleanedGoData/train/data/

1d  1k	2d  2k	3d  3k	4d  4k	5k


In [3]:
!ls

README.md	 __pycache__	  environment.yml  logs    models.py  train.py
TrainTest.ipynb  buildDataset.py  layers.py	   models  test.pt    util.py


In [12]:
bias = torch.zeros(19, 19)

In [13]:
counter = 1
for i in range(19):
    for j in range(19):
        if bias[i][j] != 0:
            continue
        bias[i][j] = counter
        bias[18 - i][j] = counter
        bias[i][18 - j] = counter
        bias[18 - i][18 - j] = counter
        bias[j][i] = counter
        bias[j][18 - i] = counter
        bias[18 - j][i] = counter
        bias[18 - j][18 - i] = counter
        counter += 1

In [15]:
361 / 8

45.125

In [59]:
torch.max(bias)

tensor(55.)

In [18]:
!ls

README.md	 __pycache__	  environment.yml  logs    models.py  train.py
TrainTest.ipynb  buildDataset.py  layers.py	   models  test.pt    util.py


In [23]:
!pip install tensorboard

     |████████████████████████████████| 6.0 MB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 52.1 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 48.7 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 48.9 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 47.1 MB/s eta 0:00:01


In [62]:
y[ind[0][0]]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [61]:
ind = torch.zeros((1, 1), dtype = torch.int32)

In [66]:
loss_fn = torch.nn.CrossEntropyLoss()

In [82]:
y.shape

torch.Size([128])

In [101]:
pred = torch.zeros((128, 361))
pred[:, 3] = 1

In [96]:
pred.shape

torch.Size([128, 361])

In [102]:
(torch.argmax(pred, dim = 1) == y).sum()

tensor(2)

In [99]:
torch.argmax(pred, dim = 1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [93]:
y

tensor([ 28, 198,   9, 119, 321,  19,  56, 302, 283, 287,  99, 271,  93, 314,
        115,  41, 315, 239,  66, 321,  93, 282, 239, 169,  40, 247, 154, 258,
        285, 264, 180,  44, 143, 108,  96, 288, 124,  60, 104,  85, 328, 240,
        338,  32,  88, 270, 167, 193,  81,  38, 211, 261,  61,  58, 274,  62,
         51, 181, 312,  86, 224, 312, 206,   3, 187, 232, 260,  24,  23, 178,
        216, 267, 196, 338,  73, 197,  50,  43, 318,  14, 272,   3, 118,  87,
        144, 179, 327,  26, 158, 185, 137,  79,  28, 235, 184, 317, 191,  49,
        265, 288, 261, 123,  70, 242, 297,  79, 109, 277, 319,  90, 108, 160,
        153, 155, 198, 128, 146, 242,  66, 129, 287, 271,  53, 169, 125,  42,
         40, 149])

In [98]:
pred

tensor([[28., 28., 28.,  ..., 28., 28., 28.],
        [28., 28., 28.,  ..., 28., 28., 28.],
        [28., 28., 28.,  ..., 28., 28., 28.],
        ...,
        [28., 28., 28.,  ..., 28., 28., 28.],
        [28., 28., 28.,  ..., 28., 28., 28.],
        [28., 28., 28.,  ..., 28., 28., 28.]])

In [115]:
with tq(total = 50000) as progress_bar:
    for i in range(500):
        progress_bar.update(100)
        progress_bar.set_postfix(epoch=i + 1)
        with tq(total=1000, leave = False) as progress_bar2:
            for j in range(10):
                progress_bar2.update(100)
                progress_bar2.set_postfix(acc = j)

 60%|██████    | 600/1000 [00:00<00:00, 54020.14it/s, acc=5]


 20%|██        | 200/1000 [00:00<00:00, 71434.97it/s, acc=1] 


100%|██████████| 1000/1000 [00:00<00:00, 49923.28it/s, acc=9]


 60%|██████    | 600/1000 [00:00<00:00, 54498.61it/s, acc=5]


 20%|██        | 200/1000 [00:00<00:00, 70682.57it/s, acc=1] 


...

 20%|██        | 200/1000 [00:00<00:00, 78390.88it/s, acc=1] 


100%|██████████| 1000/1000 [00:00<00:00, 52993.85it/s, acc=9]


 60%|██████    | 600/1000 [00:00<00:00, 45846.07it/s, acc=5]


 46%|████▌     | 22900/50000 [00:07<00:08, 3042.96it/s, epoch=229]


KeyboardInterrupt: 

In [118]:
print("\ntest")


test


In [2]:
#!python train.py test

Tensorboard logs will be saved here:  logs/test
using device:  cuda:0
model will be saved here:  models/test
getting paths
100%|████████████████████████████████| 30736/30736 [00:00<00:00, 1274164.36it/s]
building dataset
building dataloader
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch  1 
-----------------------------
  2%|▏            | 50048/2885727 [06:18<7:42:08, 102.26it/s, NLL=4.86, epoch=1]
evaluating model...

 60%|████████████████▏          | 6016/10000 [00:17<00:09, 424.19it/s, NLL=4.98]


100%|█████████████████████████████| 10000/10000 [00:26<00:00, 377.15it/s, NLL=5]
new best acc of  tensor(0.0662, device='cuda:0') Saving model at:  models/test50048
  3%|▍           | 100096/2885727 [12:28<5:46:36, 133.95it/s, NLL=5.04, epoch=1]
evaluating model...

 36%|█████████▎                | 3584/10000 [00:03<00:05, 1277.57it/s, NLL=4.99]


  5%|▌           | 150144/2885727 [18:46<7:28:18, 101.70it/s, NLL=5.14, epoch=1]
evaluating model...

 17%|████▎                     | 1664/10000 [00:01<00:05, 1400.86it/s, NLL=5.02]


100%|████████████████████████████| 10000/10000 [00:06<00:00, 2622.69it/s, NLL=5]


  7%|▊           | 200192/2885727 [25:02<4:06:19, 181.70it/s, NLL=4.95, epoch=1]
evaluating model...

 79%|████████████████████▋     | 7936/10000 [00:05<00:01, 1523.56it/s, NLL=4.99]


...

 10%|█▏          | 300288/2885727 [37:14<4:24:51, 162.69it/s, NLL=5.08, epoch=1]
evaluating model...

 41%|███████████▉                 | 4096/10000 [00:03<00:04, 1438.88it/s, NLL=5]


 12%|█▊             | 350336/2885727 [43:21<6:14:39, 112.79it/s, NLL=5, epoch=1]
evaluating model...

 23%|█████▉                    | 2304/10000 [00:02<00:05, 1342.21it/s, NLL=5.01]


 14%|█▋          | 400384/2885727 [49:25<5:34:56, 123.67it/s, NLL=4.98, epoch=1]
evaluating model...

  5%|█▌                             | 512/10000 [00:00<00:13, 694.57it/s, NLL=5]


 87%|██████████████████████▋   | 8704/10000 [00:06<00:00, 1601.44it/s, NLL=4.99]


 16%|█▊          | 447616/2885727 [55:06<5:00:10, 135.37it/s, NLL=5.13, epoch=1]
Traceback (most recent call last):
  File "train.py", line 134, in <module>
    main()
  File "train.py", line 66, in main
    for batch, (X, y) in enumerate(train_loader):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 517, in __next__
    data = self._next_data()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1182, in _next_data
    idx, data = self._get_data()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1148, in _get_data
    success, data = self._try_get_data()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 986, in _try_get_data
    data = self._data_queue.get(timeout=timeout)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", l

In [20]:
# !python train.py testConvNoRes

Tensorboard logs will be saved here:  logs/testConvNoRes
using device:  cuda:0
model will be saved here:  models/testConvNoRes
getting paths
100%|████████████████████████████████| 30736/30736 [00:00<00:00, 1197704.56it/s]
building dataset
building dataloader
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch  1 
-----------------------------
  2%|▏            | 50048/2885727 [06:01<4:50:39, 162.60it/s, NLL=3.38, epoch=1]
evaluating model...

 58%|████████████████▏           | 5760/10000 [00:19<00:09, 449.89it/s, NLL=3.1]


100%|██████████████████████████| 10000/10000 [00:28<00:00, 351.06it/s, NLL=3.09]
new best acc of  tensor(0.3251, device='cuda:0') Saving model at:  models/testConvNoRes50048
  3%|▍             | 100096/2885727 [12:48<8:44:34, 88.50it/s, NLL=2.4, epoch=1]
evaluating model...

 28%|███████▌                   | 2816/10000 [00:08<00:18, 389.81it/s, NLL=2.55]


100%|██████████████████████████▉| 9984/10000 [00:22<00:00, 401.80it/s, NLL=2.55]


100%|██████████████████████████| 10000/10000 [00:22<00:00, 444.44it/s, NLL=2.55]
new best acc of  tensor(0.3826, device='cuda:0') Saving model at:  models/testConvNoRes100096
  5%|▌           | 150144/2885727 [19:46<5:08:04, 147.99it/s, NLL=2.73, epoch=1]
evaluating model...

 73%|██████████████████▉       | 7296/10000 [00:06<00:02, 1275.70it/s, NLL=2.51]


100%|██████████████████████████| 10000/10000 [00:08<00:00, 1123.81it/s, NLL=2.5]
new best acc of  tensor(0.3991, device='cuda:0') Saving model at:  models/testConvNoRes150144
  7%|▉            | 200192/2885727 [26:28<6:30:53, 114.51it/s, NLL=2.5, epoch=1]
evaluating model...

 44%|███████████▎              | 4352/10000 [00:04<00:04, 1271.33it/s, NLL=2.48]


...

100%|██████████████████████████| 10000/10000 [00:12<00:00, 778.77it/s, NLL=2.32]
new best acc of  tensor(0.4192, device='cuda:0') Saving model at:  models/testConvNoRes400384
 16%|█▊          | 450432/2885727 [59:20<4:36:19, 146.89it/s, NLL=2.03, epoch=1]
evaluating model...

 65%|█████████████████▋         | 6528/10000 [00:13<00:04, 744.47it/s, NLL=2.27]


100%|███████████████████████████| 10000/10000 [00:18<00:00, 547.18it/s, NLL=2.3]
new best acc of  tensor(0.4239, device='cuda:0') Saving model at:  models/testConvNoRes450432
 17%|█▉         | 500480/2885727 [1:05:57<7:06:59, 93.10it/s, NLL=2.32, epoch=1]
evaluating model...

 37%|██████████                 | 3712/10000 [00:07<00:10, 625.88it/s, NLL=2.29]


100%|██████████████████████████| 10000/10000 [00:15<00:00, 649.02it/s, NLL=2.24]
new best acc of  tensor(0.4262, device='cuda:0') Saving model at:  models/testConvNoRes500480
 19%|█▉        | 550528/2885727 [1:12:35<3:43:54, 173.82it/s, NLL=2.13, epoch=1]
evaluating model...

 15%|████▏                      | 1536/10000 [00:04<00:16, 514.99it/s, NLL=2.23]


 97%|██████████████████████████▎| 9728/10000 [00:17<00:00, 765.89it/s, NLL=2.24]


100%|██████████████████████████| 10000/10000 [00:17<00:00, 557.80it/s, NLL=2.25]
new best acc of  tensor(0.4295, device='cuda:0') Saving model at:  models/testConvNoRes550528
 19%|█▉        | 552704/2885727 [1:13:05<6:25:13, 100.94it/s, NLL=2.37, epoch=1]^C


In [21]:
!python train.py testConvWRes

Tensorboard logs will be saved here:  logs/testConvWRes
using device:  cuda:0
model will be saved here:  models/testConvWRes
getting paths
100%|████████████████████████████████| 30736/30736 [00:00<00:00, 1185261.27it/s]
building dataset
building dataloader
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch  1 
-----------------------------
  2%|▏            | 50048/2885727 [06:00<5:56:44, 132.48it/s, NLL=2.64, epoch=1]
evaluating model...

 51%|██████████████▎             | 5120/10000 [00:10<00:07, 615.74it/s, NLL=2.5]


100%|██████████████████████████| 10000/10000 [00:19<00:00, 506.75it/s, NLL=2.55]
new best acc of  tensor(0.3818, device='cuda:0') Saving model at:  models/testConvWRes/50048.pth
  3%|▍           | 100096/2885727 [12:13<4:37:29, 167.31it/s, NLL=2.41, epoch=1]
evaluating model...

 22%|██████                      | 2176/10000 [00:05<00:16, 479.70it/s, NLL=2.4]


 93%|█████████████████████████▏ | 9344/10000 [00:15<00:00, 722.08it/s, NLL=2.44]


100%|██████████████████████████| 10000/10000 [00:16<00:00, 595.58it/s, NLL=2.43]
new best acc of  tensor(0.4058, device='cuda:0') Saving model at:  models/testConvWRes/100096.pth
  5%|▌           | 150144/2885727 [18:21<4:33:34, 166.66it/s, NLL=2.44, epoch=1]
evaluating model...

 59%|███████████████▎          | 5888/10000 [00:07<00:03, 1058.49it/s, NLL=2.32]


100%|██████████████████████████| 10000/10000 [00:12<00:00, 820.57it/s, NLL=2.32]
new best acc of  tensor(0.4134, device='cuda:0') Saving model at:  models/testConvWRes/150144.pth
  7%|▉            | 200192/2885727 [24:24<4:21:52, 170.92it/s, NLL=2.4, epoch=1]
evaluating model...

 20%|█████▌                     | 2048/10000 [00:02<00:06, 1144.82it/s, NLL=2.3]


...

 16%|█▊          | 450432/2885727 [55:28<3:50:51, 175.81it/s, NLL=2.37, epoch=1]
evaluating model...

 69%|██████████████████▋        | 6912/10000 [00:21<00:08, 363.54it/s, NLL=2.17]


100%|██████████████████████████| 10000/10000 [00:28<00:00, 351.74it/s, NLL=2.17]
new best acc of  tensor(0.4397, device='cuda:0') Saving model at:  models/testConvWRes/450432.pth
 17%|█▋        | 500480/2885727 [1:01:49<3:44:34, 177.02it/s, NLL=2.11, epoch=1]
evaluating model...

 38%|██████████▎                | 3840/10000 [00:13<00:15, 391.26it/s, NLL=2.12]


100%|██████████████████████████| 10000/10000 [00:27<00:00, 358.73it/s, NLL=2.12]
new best acc of  tensor(0.4468, device='cuda:0') Saving model at:  models/testConvWRes/500480.pth
 19%|█▉        | 550528/2885727 [1:08:17<4:08:13, 156.80it/s, NLL=2.08, epoch=1]
evaluating model...

  6%|█▊                          | 640/10000 [00:05<01:10, 133.37it/s, NLL=2.19]


 78%|█████████████████████      | 7808/10000 [00:23<00:04, 517.98it/s, NLL=2.14]


 20%|█▉        | 567040/2885727 [1:10:48<5:44:38, 112.13it/s, NLL=2.06, epoch=1]^C


In [ ]:
# !python train.py convFull